# Part 2 - Grid as a Secure MLaaS

<p align="center">
<img height="800px" width="800px" src="./images/secure_mlaas.png">
</p>

As mentioned in the previous section, we can use the Grid platform to run **encrypted models** and **perform inferences using encrypted input data**.  

&nbsp;Therefore, the Grid platform can work as an agnostic cloud service that can manage encrypted models stored and distributed over the cloud provider's infrastructure, protecting intellectual property and preserving user's privacy at the same time.  

Authors:
 - Ionésio Junior - Github: [IonesioJunior](https://github.com/IonesioJunior)


&nbsp; In this section, we'll learn **how to host and serve MPC models on the grid platform**.

**NOTE:** At the time of running this notebook, we were running the grid components in background mode.  

Components:
 - Grid Gateway(http://localhost:5000)
 - Grid Node Bob ( ws://localhost:3000 )
 - Grid Node Alice ( ws://localhost:3001 )
 - Grid Node Bill ( ws://localhost:3002 )
 - Grid Node James ( ws://localhost:3003 )

#### Importing dependencies

In [1]:
import syft as sy
import torch as th
import torch.nn.functional as F
from syft.workers.node_client import NodeClient

hook = sy.TorchHook(th)

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'F:\Anaconda3\envs\tfe\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


##  2.1 - Host a mpc model

#### 2.1.1 - Building our model
To simplify our example, we'll use a toy model that has not been trained previously.

In [2]:
sy.hook.local_worker.is_client_worker = False # We need to set up this flag to save plans states during model's build.

# Your model needs to extends syft plans
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__(id="convnet") # ID used to identify this model 
        self.fc1 = th.tensor([2.0, 4.0])
        self.bias = th.tensor([1000.0])

    def forward(self, x):
        return self.fc1.matmul(x) + self.bias

# Create an instance of it.
model = Net()

# Build model's plan sending any valid input (input's shape needs to match with model's dimensions)
model.build(th.tensor([1.0, 2]))

#### 2.1.2 - Hosting at  Grid
As a company, after creating a model, we want to publish it on the cloud provider's infrastructure in a secure way.

In [3]:
node1 = NodeClient(hook, "ws://localhost:3000")
node2 = NodeClient(hook, "ws://localhost:3001")
node3 = NodeClient(hook, "ws://localhost:3002")
node4 = NodeClient(hook, "ws://localhost:3003")

my_grid = sy.PrivateGridNetwork(node1,node2,node3,node4)
my_grid = sy.PublicGridNetwork(hook, "http://localhost:5000")
my_grid.serve_model(model,id=model.id,allow_remote_inference=True, mpc=True) 

# We'll use the grid platform as a cloud service.
#cloud_grid_service = sy.PublicGridNetwork(hook, "http://localhost:5000")

'''
This method will split your model weights into pieces,
distributing them through the grid network,
and storing a pointer plan that manages all remote references.
'''
#cloud_grid_service.serve_model(model,id=model.id,allow_remote_inference=True, mpc=True) # If mpc flag is False, It will host a unencrypted model.

Websocket connection closed (worker: Bill)
Created new websocket connection


RuntimeError: Websocket connection closed and creation of new connection failed.

It's important to note that the **cloud_grid_service does not have access to the real model**. It has only mpc slices distributed over the platform.  

**In a cyber-attack scenario, the malicious agent must invade all nodes involved in the mpc process to retrieve the original model.**

## 2.2 - Running encrypted inferences

As a user, we want to run inferences on our private data without sending them to anyone.  

PS: We will run it on this notebook, but in a real environment, this section needs to be implemented using syft.js library!  
The user data should never be sent over the wire, so front-end will be the best place to do it **(the private data should be encrypted on the user's device)**.

In [5]:
# Private user data that needs to be protected
user_input_data = th.tensor([5.0, 3])

'''
This method will search the desired encrypted model,
split your private data and send their slices
to the same nodes that stores the mpc model weights,
perform a distributed computing between mpc weights and mpc input data,
receive the mpc results and aggregate it, returning the inference's result.
'''
result = cloud_grid_service.run_remote_inference("convnet", user_input_data, mpc=True)# If mpc flag is False, It will send your real data to the platform.
print("Inference's result: ", result) # ( [2.0, 4.0] * [5.0, 3.0] ) + [1000] = [1022]

Inference's result:  tensor([1022.])


# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on GitHub

The easiest way to help our community is just by starring the GitHub repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)